DATA PREPARATION

In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import cv2

BASE_DIR = Path.cwd()
DATASET_PATH = BASE_DIR / "dataset_filtered"
LABEL_MAP = BASE_DIR / "label_map.txt"
MODEL_NAME = BASE_DIR / "crop_disease_model.keras"
EPOCHS = 12

In [2]:
if not DATASET_PATH.exists():
    raise FileNotFoundError(
        f"Dataset folder not found at {DATASET_PATH}\n"
        "Please download the dataset and place it in the project root as 'dataset_filtered/'"
    )

In [3]:
train_datagen = ImageDataGenerator(
    preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2,
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [4]:
val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

In [5]:
train_data = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="training"
)

Found 16254 images belonging to 13 classes.


In [6]:
val_data = val_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)

Found 4058 images belonging to 13 classes.


In [7]:
NUM_CLASSES = train_data.num_classes
labels = train_data.class_indices
with open(LABEL_MAP, "w") as f:
    for label, idx in labels.items():
        f.write(f"{idx}:{label}\n")

print("✅ Label map saved to", LABEL_MAP)

✅ Label map saved to d:\Lohith\BTECH CODES\Machine_Learning\Project\Crop-Disease\label_map.txt


MODEL ARCHITECTURE AND TRAINING

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False 

In [9]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")
])

In [10]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 13)             │         3,341 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,589,261 (9.88 MB)

 Trainable params: 331,277 (1.26 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [12]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_data.classes),
    y=train_data.classes
)

class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)

Class Weights: {0: np.float64(1.5628846153846154), 1: np.float64(1.5628846153846154), 2: np.float64(10.2484237074401), 3: np.float64(0.7346108650456477), 4: np.float64(1.5628846153846154), 5: np.float64(0.8182641965364479), 6: np.float64(1.6408237431859478), 7: np.float64(0.8823625210357744), 8: np.float64(0.9323696437790397), 9: np.float64(1.1123733917328222), 10: np.float64(0.29171901360422126), 11: np.float64(4.181631077952148), 12: np.float64(0.9821741494954378)}


In [13]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(MODEL_NAME, save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.3)
]

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    class_weight=class_weights,
    callbacks=callbacks
)
print("Final Training Accuracy:", history.history["accuracy"][-1])

Epoch 1/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 606ms/step - accuracy: 0.5200 - loss: 1.6086

508/508 ━━━━━━━━━━━━━━━━━━━━ 360s 705ms/step - accuracy: 0.6617 - loss: 1.1069 - val_accuracy: 0.8339 - val_loss: 0.5182 - learning_rate: 3.0000e-04
Epoch 2/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step - accuracy: 0.8057 - loss: 0.6131

508/508 ━━━━━━━━━━━━━━━━━━━━ 203s 400ms/step - accuracy: 0.8160 - loss: 0.5764 - val_accuracy: 0.8583 - val_loss: 0.4111 - learning_rate: 3.0000e-04
Epoch 3/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step - accuracy: 0.8498 - loss: 0.4712

508/508 ━━━━━━━━━━━━━━━━━━━━ 218s 429ms/step - accuracy: 0.8513 - loss: 0.4636 - val_accuracy: 0.8746 - val_loss: 0.3777 - learning_rate: 3.0000e-04
Epoch 4/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step - accuracy: 0.8584 - loss: 0.4245

508/508 ━━━━━━━━━━━━━━━━━━━━ 209s 411ms/step - accuracy: 0.8626 - loss: 0.4192 - val_accuracy: 0.8795 - val_loss: 0.3434 - learning_rate: 3.0000e-04
Epoch 5/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step - accuracy: 0.8719 - loss: 0.3951

...

508/508 ━━━━━━━━━━━━━━━━━━━━ 280s 551ms/step - accuracy: 0.8888 - loss: 0.3454 - val_accuracy: 0.8844 - val_loss: 0.3272 - learning_rate: 3.0000e-04
Epoch 7/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step - accuracy: 0.8941 - loss: 0.3240

508/508 ━━━━━━━━━━━━━━━━━━━━ 198s 389ms/step - accuracy: 0.8935 - loss: 0.3261 - val_accuracy: 0.8928 - val_loss: 0.3069 - learning_rate: 3.0000e-04
Epoch 8/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step - accuracy: 0.8981 - loss: 0.3015

508/508 ━━━━━━━━━━━━━━━━━━━━ 202s 398ms/step - accuracy: 0.8942 - loss: 0.3153 - val_accuracy: 0.9071 - val_loss: 0.2705 - learning_rate: 3.0000e-04
Epoch 9/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 196s 387ms/step - accuracy: 0.9013 - loss: 0.3007 - val_accuracy: 0.9017 - val_loss: 0.2758 - learning_rate: 3.0000e-04
Epoch 10/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step - accuracy: 0.9056 - loss: 0.2802

508/508 ━━━━━━━━━━━━━━━━━━━━ 195s 384ms/step - accuracy: 0.9076 - loss: 0.2807 - val_accuracy: 0.9073 - val_loss: 0.2656 - learning_rate: 3.0000e-04
Epoch 11/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 196s 387ms/step - accuracy: 0.9107 - loss: 0.2706 - val_accuracy: 0.8975 - val_loss: 0.3041 - learning_rate: 3.0000e-04
Epoch 12/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step - accuracy: 0.9123 - loss: 0.2569

508/508 ━━━━━━━━━━━━━━━━━━━━ 194s 381ms/step - accuracy: 0.9101 - loss: 0.2643 - val_accuracy: 0.9128 - val_loss: 0.2543 - learning_rate: 3.0000e-04


In [79]:
model.save(MODEL_NAME)
print("✅ Final model saved")


✅ Final model saved


In [65]:
label_map = {}
with open(LABEL_MAP, "r") as f:
    for line in f:
        idx, label = line.strip().split(":")
        label_map[int(idx)] = label

In [100]:
model = tf.keras.models.load_model(MODEL_NAME)

dummy_input = np.zeros((1, 224, 224, 3), dtype=np.float32)
model(dummy_input)  # 👈 IMPORTANT: call model directly, not predict()


<tf.Tensor: shape=(1, 13), dtype=float32, numpy=
array([[5.0558413e-07, 1.2270913e-06, 8.2976549e-06, 1.0191977e-03,
        5.5138069e-05, 9.8553503e-01, 1.2282942e-05, 1.1671874e-02,
        4.0866922e-05, 1.3072156e-05, 4.4176888e-04, 5.9067814e-05,
        1.1416340e-03]], dtype=float32)>

In [127]:
def is_blurry(img_path):
    img = cv2.imread(img_path)
    variance = cv2.Laplacian(img, cv2.CV_64F).var()
    threshold = 100.0  # You can adjust this threshold
    return variance < threshold

In [128]:
def generate_gradcam(model, img_path, layer_name=None):
    """Generate a Grad-CAM heatmap overlay for the predicted class.

    Fixes Keras graph issues by building the Grad-CAM model using the
    MobileNetV2 base model's input graph (not Sequential's symbolic tensors).
    """
    base_model = model.layers[0]  # MobileNetV2 (Functional)

    # Pick a sensible conv layer if not provided
    if layer_name is None:
        for layer in reversed(base_model.layers):
            if isinstance(layer, tf.keras.layers.Conv2D):
                layer_name = layer.name
                break
        if layer_name is None:
            # Fallback: any layer with 'conv' in its name
            for layer in reversed(base_model.layers):
                if "conv" in layer.name.lower():
                    layer_name = layer.name
                    break

    if layer_name is None:
        raise ValueError("No convolutional layer found in base model.")

    conv_layer = base_model.get_layer(layer_name)

    # Build a Grad-CAM graph rooted at base_model.input to avoid tensor mismatch
    x = base_model.output
    for head_layer in model.layers[1:]:
        x = head_layer(x)
    preds = x

    grad_model = tf.keras.models.Model(
        inputs=base_model.input,
        outputs=[conv_layer.output, preds],
    )

    # Load and preprocess image
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0).astype(np.float32)

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        loss = predictions[:, pred_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(pooled_grads * conv_outputs, axis=-1)

    # Convert to numpy once, then use numpy/cv2 safely
    heatmap = heatmap.numpy()
    heatmap = np.maximum(heatmap, 0)
    heatmap /= (np.max(heatmap) + 1e-8)

    heatmap = cv2.resize(heatmap, (224, 224))
    heatmap = np.uint8(255 * heatmap)

    original = cv2.imread(img_path)
    if original is None:
        raise FileNotFoundError(f"Could not read image from path: {img_path}")
    original = cv2.resize(original, (224, 224))

    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(original, 0.6, heatmap, 0.4, 0)

    output_path = "heatmap_output.jpg"
    cv2.imwrite(output_path, overlay)
    return output_path

In [129]:
def predict_disease(img_path, confidence_threshold=0.45):
    if is_blurry(img_path):
        return {
            "status": "error",
            "message": "Image is too blurry. Please retake photo.",
            "confidence": 0.0
        }
    
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    preds = model.predict(img_array)[0]
    top_idx = np.argmax(preds)
    confidence = float(preds[top_idx])
    disease_name = label_map[top_idx]

    # Epic 3: Unknown/Unclear Result Story
    if confidence < confidence_threshold:
        return {
            "status": "unknown",
            "disease": "Unknown/Unclear",
            "message": "Please consult a human expert or retake the photo.",
            "confidence": round(confidence * 100, 2)
        }
    
    # Epic 3: Healthy Plant Story
    # Check if the predicted label contains the word "healthy"
    is_healthy = "healthy" in disease_name.lower()

    return {
        "status": "success",
        "disease": disease_name,
        "is_healthy": is_healthy,
        "confidence": round(confidence * 100, 2)
    }

In [130]:
print("Final Training Accuracy:", history.history["accuracy"][-1])
print("Final Validation Accuracy:", history.history["val_accuracy"][-1])


Final Training Accuracy: 0.9100528955459595
Final Validation Accuracy: 0.9127649068832397


In [131]:
def predict_top_k(img_path, k=3, min_conf=0.05):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    preds = model.predict(img_array)[0]
    top_indices = preds.argsort()[-k:][::-1]

    results = []
    for idx in top_indices:
        conf = float(preds[idx])
        if conf >= min_conf:
            results.append({
                "disease": label_map[idx],
                "confidence": round(conf * 100, 2)
            })

    return results


In [132]:
def estimate_severity_from_heatmap(heatmap_path):
    heatmap = cv2.imread(heatmap_path)
    heatmap_gray = cv2.cvtColor(heatmap, cv2.COLOR_BGR2GRAY)

    # Threshold to isolate "hot" infected zones
    _, binary_map = cv2.threshold(heatmap_gray, 200, 255, cv2.THRESH_BINARY)

    infected_area = np.sum(binary_map == 255)
    total_area = binary_map.shape[0] * binary_map.shape[1]

    ratio = infected_area / total_area

    if ratio < 0.10:
        severity = "Low"
    elif ratio < 0.40:
        severity = "Medium"
    else:
        severity = "High"

    return {
        "severity": severity,
        "infected_ratio": round(ratio * 100, 2)
    }


In [121]:
def full_predict_with_heatmap(img_path):
    result = predict_disease(img_path)

    if result["status"] != "success":
        return result

    heatmap_path = generate_gradcam(model, img_path)
    severity_info = estimate_severity_from_heatmap(heatmap_path)

    result["heatmap"] = heatmap_path
    result["severity"] = severity_info["severity"]
    result["infected_ratio"] = severity_info["infected_ratio"]

    return result

In [133]:
if __name__ == "__main__":
    TEST_IMAGE = r'0a8a68ee-f587-4dea-beec-79d02e7d3fa4___RS_Early.B 8461.JPG'

    result = full_predict_with_heatmap(TEST_IMAGE)
    print("\n--- Final Prediction with Severity ---")
    print(result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step

--- Final Prediction with Severity ---
{'status': 'success', 'disease': 'Potato___Early_blight', 'is_healthy': False, 'confidence': 100.0, 'heatmap': 'heatmap_output.jpg', 'severity': 'Low', 'infected_ratio': np.float64(0.86)}
